In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [2]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [6]:
model.invoke("which country won the 2022 world cup?")

AIMessage(content='As an AI, I am unable to predict future events. The 2022 FIFA World Cup is scheduled to take place in Qatar from November 21 to December 18, 2022. The winner of the tournament has not been determined yet.', response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 17, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f9f71655-c875-444d-b830-094df40333b3-0', usage_metadata={'input_tokens': 17, 'output_tokens': 50, 'total_tokens': 67})

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("what is the day today?")

'Today is Wednesday.'